## Sentiment Analysis for Korean Movie Review Data

## 1. Load Pickle File

In [1]:
import pickle

with open("movie_data.pickle", "rb") as f:
    movie_data = pickle.load(f)

In [2]:
movie_data.keys()

dict_keys(['reviews', 'scores', 'reviews_ix', 'word2ix', 'ix2word', 'max_seq_length'])

In [3]:
reviews = movie_data["reviews"]
scores = movie_data["scores"]
reviews_ix = movie_data["reviews_ix"]
word2ix = movie_data["word2ix"]
ix2word = movie_data["ix2word"]
max_seq_length = movie_data["max_seq_length"]

In [4]:
for i, score in enumerate(scores):
    if score <= 6:
        scores[i] = 0
    else:
        scores[i] = 1

In [5]:
from collections import Counter
Counter(scores)

Counter({1: 1138, 0: 689})

## 2. Prepare Trian / Test Datasets

In [6]:
def pad_sequence(seq, max_seq_length):
    if len(seq) < max_seq_length:
        seq += [word2ix.get("<PAD>")]*(max_seq_length-len(seq))
    return seq

print(len(reviews_ix[0]))
print(len(pad_sequence(reviews_ix[0], max_seq_length)))

reviews_ix = [pad_sequence(reviews_ix[i], max_seq_length) for i in range(len(reviews_ix))] 

4
113


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(reviews_ix, 
                                                    scores, 
                                                    test_size=0.3, 
                                                    random_state=777)

In [8]:
len(reviews_ix)

1827

In [9]:
len(X_train)

1278

In [10]:
len(X_test)

549

## 3. Modeling

#### 3.4. RNN

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
torch.manual_seed(777);

In [12]:
torch.cuda.is_available()

True

In [13]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [14]:
reviews_ix[0]

[1512,
 2279,
 1491,
 4296,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [15]:
class GRU_Clf(nn.Module):
    def __init__(self, vocab_size, embedding_size):
        
        super(GRU_Clf, self).__init__()
        
        self.embed = nn.Embedding(vocab_size,
                                  embedding_size,
                                  padding_idx=word2ix.get("<PAD>"))
        
        self.gru = nn.GRU(input_size=embedding_size, 
                          hidden_size=hidden_size, 
                          num_layers=num_layers, 
                          batch_first=True,
                          dropout = 0.3,
                          bidirectional=bidirectional)

        self.linear = nn.Linear(hidden_size*num_directions, num_classes)
                    
    
    def forward(self,inputs):
        
        hidden = self.init_hidden(batch_size)
        embed = self.embed(inputs)
        # Propagate embedding through RNN
        # Input: (batch, seq_len, embedding_size)
        # hidden: (num_layers * num_directions, batch, hidden_size)
        
        out, _ = self.gru(embed, hidden)

        return self.linear(torch.mean(out, 1))

    def predict(self,inputs):
        
        hidden = self.init_hidden(1)
        embed = self.embed(inputs)

        # Propagate embedding through RNN
        # Input: (batch, seq_len, embedding_size)
        # hidden: (num_layers * num_directions, batch, hidden_size)
        
        out, _ = self.gru(embed, hidden)

        return self.linear(torch.mean(out, 1))
    
    def init_hidden(self, batch_size):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size)
        return Variable(torch.zeros(num_layers*num_directions, batch_size, hidden_size)).to(device)
    

In [16]:
EPOCHS = 5000
LR = 0.01
hidden_size = max_seq_length
num_classes = 2
num_layers = 1
num_directions = 1
bidirectional=False
batch_size = len(X_train)

model = GRU_Clf(vocab_size = len(word2ix), embedding_size = 100).to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=LR, momentum=0.9)

for epoch in range(EPOCHS):

    model.zero_grad()
    inputs = Variable(torch.LongTensor(X_train)).to(device)
    targets = Variable(torch.LongTensor(y_train)).to(device)
    
    preds = model(inputs)
    
    loss = loss_function(preds, targets)

    if epoch % 500 == 0:
        print(loss.item())
    
    loss.backward()
    optimizer.step()

/home/gyuwonable/anaconda3/envs/tf19/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


0.6860640048980713
0.6652719974517822
0.6607757806777954
0.6481375694274902
0.8337383270263672
0.5546431541442871
0.5901779532432556
0.5816429257392883
1.099953293800354
0.4775415360927582


In [17]:
correct = 0
for i, seq in enumerate(X_test):
    
    input = Variable(torch.LongTensor(seq).view(1,-1)).to(device)
    pred = model.predict(input)
    _, pred = torch.max(pred, 1)
    true = y_test[i]
    if true == pred.item():
        correct +=1
    
    if i%100 == 0:
        input_seq = [ix2word.get(ix) for ix in seq if ix != 0]
        print("Input :", input_seq)
        print("Prediction :", pred.item())
        print("Truth :", y_test[i])
        print("\n")

print("Accuracy :", (correct/len(X_test)*100))

Input : ['후반/Noun', '쫄렸다/Noun']
Prediction : 1
Truth : 1


Input : ['감동/Noun', '영화/Noun', '보고/Noun', '운/Noun', '거의/Noun', '애니메이션/Noun', '보면서/Verb', '울줄/Verb', '몰랐네요/Verb', 'ㅜ/KoreanParticle', '감동/Noun', 'ㅜㅜ/KoreanParticle']
Prediction : 1
Truth : 1


Input : ['스토리/Noun', '별로/Noun', '노래/Noun']
Prediction : 0
Truth : 1


Input : ['히어로/Noun', '물/Noun', '찍어도/Verb', '될/Verb', '정도/Noun', '그래픽/Noun', '본/Verb', '한국영/Noun', '화의/Noun', '희망/Noun', '이야기/Noun', '거기/Noun']
Prediction : 1
Truth : 0


Input : ['보통/Noun', '서로/Noun', '아는/Verb', '상황/Noun', '주먹/Noun', '메/Noun', '쳐서/Verb', '다른/Noun', '격방/Noun', '시도/Noun', '하는게/Verb', '정상/Noun', '로메/Noun', '쳐/Verb', '대는게/Verb', '인상/Noun', '또/Noun', '와칸/Noun', '다인/Noun', '가에서/Verb', '개때/Noun', '닥치는데/Verb', '굳이/Noun', '칼/Noun', '빼/Noun', '일일이/Noun', '상대/Noun', '하는것도/Verb', '졸/Noun', '인상/Noun', '과거/Noun', '마징/Noun', '가가/Noun', '싸우다가/Verb', '죽기/Verb', '직전/Noun', '가슴/Noun', '원자력/Noun', '빔/Noun', '쏴서/Verb', '이기는거/Verb', '배운듯/Verb']
Prediction : 0
Truth : 0


Inpu